In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn
import scipy
import scipy.stats as st
import string
import warnings
import pickle

import os
import glob
import random
from time import time

### function - get body part location for each pair of cameras

In [ ]:
from ana_functions.body_part_locs_eachpair import body_part_locs_eachpair

### function - find social gaze time point

In [ ]:
from ana_functions.find_socialgaze_timepoint import find_socialgaze_timepoint
from ana_functions.find_socialgaze_timepoint_Anipose import find_socialgaze_timepoint_Anipose
from ana_functions.find_socialgaze_timepoint_Anipose_2 import find_socialgaze_timepoint_Anipose_2

### function - define time point of behavioral events

In [ ]:
from ana_functions.bhv_events_timepoint import bhv_events_timepoint
from ana_functions.bhv_events_timepoint_Anipose import bhv_events_timepoint_Anipose

### function - plot behavioral events

In [ ]:
from ana_functions.plot_bhv_events import plot_bhv_events
from ana_functions.plot_bhv_events_levertube import plot_bhv_events_levertube
from ana_functions.draw_self_loop import draw_self_loop
import matplotlib.patches as mpatches 
from matplotlib.collections import PatchCollection

### function - plot inter-pull interval

In [ ]:
from ana_functions.plot_interpull_interval import plot_interpull_interval

### function - make demo videos with skeleton and inportant vectors

In [ ]:
from ana_functions.tracking_video_singlecam_demo import tracking_video_singlecam_demo
from ana_functions.tracking_video_Anipose_events_demo import tracking_video_Anipose_events_demo

### function - interval between all behavioral events

In [ ]:
from ana_functions.bhv_events_interval import bhv_events_interval

## Analyze each session

### prepare the basic behavioral data (especially the time stamps for each bhv events)
### results are based on Anipose 3d reconstruction

In [ ]:
# gaze angle threshold
# angle_thres = np.pi/36 # 5 degree
# angle_thres = np.pi/18 # 10 degree
angle_thres = np.pi/12 # 15 degree
# angle_thres = np.pi/4 # 45 degree
# angle_thres = np.pi/6 # 30 degree
angle_thres_name = '15'

merge_campairs = ['_Anipose'] # "_Anipose": this script is only for Anipose 3d reconstruction of camera 1,2,3 

with_tubelever = 1 # 1: consider the location of tubes and levers, only works if using Anipose 3d (or single camera)

# get the fps of the analyzed video
fps = 30

# frame number of the demo video
# nframes = 0.5*30 # second*30fps
nframes = 2*30 # second*30fps

# re-analyze the video or not
reanalyze_video = 0
redo_anystep = 1

# only analyze the best (five) sessions for each conditions
do_bestsession = 1
if do_bestsession:
    savefile_sufix = '_bestsessions'
else:
    savefile_sufix = ''
    
# all the videos (no misaligned ones)
# aligned with the audio
# get the session start time from "videosound_bhv_sync.py/.ipynb"
# currently the session_start_time will be manually typed in. It can be updated after a better method is used

# Koala Vermelho
if 1:
    if not do_bestsession:
        dates_list = [
                      "20230801","20230802","20230804","20230808","20230809","20230810",
                      "20230811","20230816","20230822","20230823","20230918","20231002",
                    "20230818_2","20231019","20231020","20231023","20231024","20231025",
                      "20231026","20231027","20231030","20231031","20231101","20231102",
                    # "20231106","20231107","20231109",
                     ]
        session_start_times = [ 
                                 19.5, 24.0, 20.0, 49.5, 40.8, 32.0,
                                 40.2, 22.5, 25.5, 25.8, 28.0, 25.0, 
                                 37.5, 17.2, 17.5, 17.0, 24.0, 27.5,
                                 16.5, 24.0, 17.0, 21.0, 26.0, 20.5,
                               # 14.5, 33.2, 21.0,
                              ] # in second 
    elif do_bestsession:
        dates_list = [
                      "20230801","20230802","20230804","20230808","20230809","20230810",
                      "20230811","20230816","20230822","20230823","20230918","20231002", 
                    "20230818_2","20231019","20231020","20231023","20231024","20231025",
                      "20231026","20231027","20231030","20231031","20231101","20231102",
                    # "20231106","20231107","20231109"
                     ]
        session_start_times = [ 
                                 19.5, 24.0, 20.0, 49.5, 40.8, 32.0,
                                 40.2, 22.5, 25.5, 25.8, 28.0, 25.0, 
                                 37.5, 17.2, 17.5, 17.0, 24.0, 27.5,
                                 16.5, 24.0, 17.0, 21.0, 26.0, 20.5,
                              #  14.5, 33.2, 21.0,
                              ] # in second 
    
    animal1_fixedorder = ['koala']
    animal2_fixedorder = ['vermelho']

    animal1_filename = "Koala"
    animal2_filename = "Vermelho"
    

#
# dates_list = ["20230802"]
# session_start_times = [24.0] # in second

ndates = np.shape(dates_list)[0]

session_start_frames = session_start_times * fps # fps is 30Hz

totalsess_time = 20*60

# video tracking results info
animalnames_videotrack = ['dodson','scorch'] # does not really mean dodson and scorch, instead, indicate animal1 and animal2
bodypartnames_videotrack = ['rightTuft','whiteBlaze','leftTuft','rightEye','leftEye','mouth']

# location of levers and tubes for camera 2
# get this information using DLC animal tracking GUI, the results are stored: 
# /home/ws523/marmoset_tracking_DLCv2/marmoset_tracking_with_lever_tube-weikang-2023-04-13/labeled-data/
considerlevertube = 1
considertubeonly = 0


if np.shape(session_start_times)[0] != np.shape(dates_list)[0]:
    exit()

    
# define bhv events summarizing variables     
succ_rate_all_dates = np.zeros((ndates,1))
interpullintv_all_dates = np.zeros((ndates,1))
trialnum_all_dates = np.zeros((ndates,1))
blocktime_all_dates = np.zeros((ndates,1))
totalsess_time_all_dates = np.zeros((ndates,1))
animal1_blocked_all_dates = np.zeros((ndates,1))
animal2_blocked_all_dates = np.zeros((ndates,1))
#
owgaze1_num_all_dates = np.zeros((ndates,1))
owgaze2_num_all_dates = np.zeros((ndates,1))
mtgaze1_num_all_dates = np.zeros((ndates,1))
mtgaze2_num_all_dates = np.zeros((ndates,1))
pull1_num_all_dates = np.zeros((ndates,1))
pull2_num_all_dates = np.zeros((ndates,1))
#
bhv_intv_all_dates = dict.fromkeys(dates_list, [])
#
# task 1 - competition
trialnum_task1_all_dates = np.zeros((ndates,1))
blocktime_task1_all_dates = np.zeros((ndates,1))
owgaze1_num_task1_all_dates = np.zeros((ndates,1))
owgaze2_num_task1_all_dates = np.zeros((ndates,1))
mtgaze1_num_task1_all_dates = np.zeros((ndates,1))
mtgaze2_num_task1_all_dates = np.zeros((ndates,1))
pull1_num_task1_all_dates = np.zeros((ndates,1))
pull2_num_task1_all_dates = np.zeros((ndates,1))
# task 2 - dilemma
trialnum_task2_all_dates = np.zeros((ndates,1))
blocktime_task2_all_dates = np.zeros((ndates,1))
owgaze1_num_task2_all_dates = np.zeros((ndates,1))
owgaze2_num_task2_all_dates = np.zeros((ndates,1))
mtgaze1_num_task2_all_dates = np.zeros((ndates,1))
mtgaze2_num_task2_all_dates = np.zeros((ndates,1))
pull1_num_task2_all_dates = np.zeros((ndates,1))
pull2_num_task2_all_dates = np.zeros((ndates,1))



# where to save the summarizing data
data_saved_folder = '/gpfs/gibbs/pi/jadi/VideoTracker_SocialInter/3d_recontruction_analysis_joystick_task_data_saved/'
if not os.path.exists(data_saved_folder):
    os.makedirs(data_saved_folder)

# where to save the demo video
withboxCorner = 0
video_file_dir = data_saved_folder+'/example_videos_Anipose_bhv_demo/'+animal1_filename+'_'+animal2_filename
if not os.path.exists(video_file_dir):
    os.makedirs(video_file_dir)
    
    

In [ ]:
# basic behavior analysis (define time stamps for each bhv events, etc)

try:
    if redo_anystep:
        dummy
    
    # load saved data
    data_saved_subfolder = data_saved_folder+'data_saved_Anipose'+savefile_sufix+'/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
    
    with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/totalsess_time_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        totalsess_time_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/blocktime_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        blocktime_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/animal1_blocked_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        animal1_blocked_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/animal2_blocked_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        animal2_blocked_all_dates = pickle.load(f)
        
    #
    with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        succ_rate_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        interpullintv_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        trialnum_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        bhv_intv_all_dates = pickle.load(f)
        
    # load task 1 data
    with open(data_saved_subfolder+'/owgaze1_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze1_num_task1_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/owgaze2_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze2_num_task1_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze1_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze1_num_task1_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze2_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze2_num_task1_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull1_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull1_num_task1_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull2_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull2_num_task1_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/blocktime_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        blocktime_task1_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/trialnum_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        trialnum_task1_all_dates = pickle.load(f)
        
    # load task 2 data
    with open(data_saved_subfolder+'/owgaze1_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze1_num_task2_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/owgaze2_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        owgaze2_num_task2_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze1_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze1_num_task2_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze2_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        mtgaze2_num_task2_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull1_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull1_num_task2_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull2_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        pull2_num_task2_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/blocktime_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        blocktime_task2_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/trialnum_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'rb') as f:
        trialnum_task2_all_dates = pickle.load(f)

    print('all data from all dates are loaded')

except:

    print('analyze all dates')

    for idate in np.arange(0,ndates,1):
        date_tgt = dates_list[idate]
        session_start_time = session_start_times[idate]

        # folder and file path
        camera12_analyzed_path = "/gpfs/gibbs/pi/jadi/VideoTracker_SocialInter/test_video_joystick_task_3d/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_camera12/"
        Anipose_analyzed_path = "/gpfs/gibbs/pi/jadi/VideoTracker_SocialInter/test_video_joystick_task_3d/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_camera12/anipose_cam123_3d_h5_files/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
 
        # should be only one merge type - "Anipose"
        merge_campair = merge_campairs[0]
    
        # load camera tracking results
        try:
            dummy
            if reanalyze_video:
                print("re-analyze the data ",date_tgt)
                dummy
            ## read
            with open(Anipose_analyzed_path + 'body_part_locs_Anipose.pkl', 'rb') as f:
                body_part_locs_Anipose = pickle.load(f)                 
        except:
            print("did not save data for Anipose - body part tracking "+date_tgt)
            # analyze and save
            Anipose_h5_file = Anipose_analyzed_path +date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_anipose.h5"
            Anipose_h5_data = pd.read_hdf(Anipose_h5_file)
            body_part_locs_Anipose = body_part_locs_eachpair(Anipose_h5_data)
            with open(Anipose_analyzed_path + 'body_part_locs_Anipose.pkl', 'wb') as f:
                pickle.dump(body_part_locs_Anipose, f)            

        min_length = np.min(list(body_part_locs_Anipose.values())[0].shape[0])
        
        
        # load behavioral results
        try:
            bhv_data_path = "/home/ws523/marmoset_tracking_bhv_data_joystick_task/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
            trial_record_json = glob.glob(bhv_data_path +date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_TrialRecord_" + "*.json")
            bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_bhv_data_" + "*.json")
            session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_session_info_" + "*.json")
            #
            trial_record = pd.read_json(trial_record_json[0])
            bhv_data = pd.read_json(bhv_data_json[0])
            session_info = pd.read_json(session_info_json[0])
        except:
            bhv_data_path = "/home/ws523/marmoset_tracking_bhv_data_joystick_task/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
            trial_record_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_TrialRecord_" + "*.json")
            bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_bhv_data_" + "*.json")
            session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_session_info_" + "*.json")
            #
            trial_record = pd.read_json(trial_record_json[0])
            bhv_data = pd.read_json(bhv_data_json[0])
            session_info = pd.read_json(session_info_json[0])

        # get animal info from the session information
        animal1 = session_info['animal1_name'][0].lower()
        animal2 = session_info['animal2_name'][0].lower()
        
        # if one of the animals is blocked from the lever
        try:
            animal1_block = session_info['animal1_block'][0]
            animal2_block = session_info['animal2_block'][0]
        except:
            animal1_block = 0
            animal2_block = 0
        #
        if animal1 == animal1_fixedorder[0]:
            animal1_blocked_all_dates[idate] = animal1_block
            animal2_blocked_all_dates[idate] = animal2_block
        elif animal2 == animal1_fixedorder[0]:
            animal1_blocked_all_dates[idate] = animal2_block
            animal2_blocked_all_dates[idate] = animal1_block
            
        # 
        highvalue = session_info['large_reward_volume'][0]
        lowvalue = session_info['small_reward_volume'][0]

        # clean up the trial_record
        warnings.filterwarnings('ignore')
        trial_record_clean = pd.DataFrame(columns=trial_record.columns)
        # for itrial in np.arange(0,np.max(trial_record['trial_number']),1):
        for itrial in list(trial_record['trial_number']-1):
            # trial_record_clean.loc[itrial] = trial_record[trial_record['trial_number']==itrial+1].iloc[[0]]
            trial_record_clean = trial_record_clean.append(trial_record[trial_record['trial_number']==itrial+1].iloc[[0]])
        trial_record_clean = trial_record_clean.reset_index(drop = True)

        # change bhv_data time to the absolute time
        time_points_new = pd.DataFrame(np.zeros(np.shape(bhv_data)[0]),columns=["time_points_new"])
        # for itrial in np.arange(0,np.max(trial_record_clean['trial_number']),1):
        for itrial in list(trial_record['trial_number']-1):
            ind = bhv_data["trial_number"]==itrial+1
            new_time_itrial = bhv_data[ind]["time_points"] + trial_record_clean["trial_starttime"].iloc[itrial]
            time_points_new["time_points_new"][ind] = new_time_itrial
        bhv_data["time_points"] = time_points_new["time_points_new"]
        bhv_data = bhv_data[bhv_data["time_points"] != 0]

        # get the tasktype pertrial and separate the time for each task type
        tasktype_pertrial = trial_record_clean['task_type']
        tasktype_unique = np.unique(tasktype_pertrial)
        if np.shape(tasktype_unique)[0]>1:
            ifAlterTask = 1 # this session has alternating task type
            #
            # trial for task 1 - competition
            ind_task1 = tasktype_pertrial == np.sort(np.unique(tasktype_pertrial))[0]
            trialnum_task1 = trial_record_clean['trial_number'][ind_task1].reset_index(drop=True)
            bhv_data_task1 = bhv_data[np.isin(bhv_data["trial_number"],trialnum_task1)]
            #
            nsemiblocks_task1 = np.sum(trialnum_task1.diff()>1) + 1
            semiblocks_starttrial_task1 = np.append(np.array([trialnum_task1.iloc[0]]),np.array(trialnum_task1[trialnum_task1.diff()>1]))
            semiblocks_endtrial_task1 = np.append(np.array(trialnum_task1[trialnum_task1.diff(-1)<-1]),np.array([trialnum_task1.iloc[-1]]))
            # semiblock time (align with the trial start)
            semiblocks_period_task1 = np.zeros((nsemiblocks_task1 ,2))
            for isemiblock in np.arange(0,nsemiblocks_task1,1):
                # start time 
                bhv_data_starttrial = bhv_data_task1[bhv_data_task1['trial_number']==semiblocks_starttrial_task1[isemiblock]]
                semiblocks_period_task1[isemiblock,0] = bhv_data_starttrial[bhv_data_starttrial['behavior_events']==0]['time_points']
                # end time 
                bhv_data_endtrial = bhv_data_task1[bhv_data_task1['trial_number']==semiblocks_endtrial_task1[isemiblock]]
                semiblocks_period_task1[isemiblock,1] = bhv_data_endtrial[bhv_data_endtrial['behavior_events']==9]['time_points']
            #
            # trial for task 2 - dilemma
            ind_task2 = tasktype_pertrial == np.sort(np.unique(tasktype_pertrial))[1]
            trialnum_task2 = trial_record_clean['trial_number'][ind_task2].reset_index(drop=True)
            bhv_data_task2 = bhv_data[np.isin(bhv_data["trial_number"],trialnum_task2)]
            #
            nsemiblocks_task2 = np.sum(trialnum_task2.diff()>1) + 1
            semiblocks_starttrial_task2 = np.append(np.array([trialnum_task2.iloc[0]]),np.array(trialnum_task2[trialnum_task2.diff()>1]))
            semiblocks_endtrial_task2 = np.append(np.array(trialnum_task2[trialnum_task2.diff(-1)<-1]),np.array([trialnum_task2.iloc[-1]]))
            # semiblock time (align with the trial start)
            semiblocks_period_task2 = np.zeros((nsemiblocks_task2 ,2))
            for isemiblock in np.arange(0,nsemiblocks_task2,1):
                # start time 
                bhv_data_starttrial = bhv_data_task2[bhv_data_task2['trial_number']==semiblocks_starttrial_task2[isemiblock]]
                semiblocks_period_task2[isemiblock,0] = bhv_data_starttrial[bhv_data_starttrial['behavior_events']==0]['time_points']
                # end time 
                bhv_data_endtrial = bhv_data_task2[bhv_data_task2['trial_number']==semiblocks_endtrial_task2[isemiblock]]
                semiblocks_period_task2[isemiblock,1] = bhv_data_endtrial[bhv_data_endtrial['behavior_events']==9]['time_points']
            #
            trialnum_task1_all_dates[idate] = np.shape(trialnum_task1)[0]
            trialnum_task2_all_dates[idate] = np.shape(trialnum_task2)[0]
      
        
        # use the total session time as the total time
        totalsess_time = np.ceil(np.max(bhv_data['time_points']))
        totalsess_time_all_dates[idate] = totalsess_time
        if not ifAlterTask:
            blocktime_all_dates[idate] = totalsess_time
            
        # get the time of each task type
        if ifAlterTask:
            semiblock_time_task1 = 0
            for isemiblock in np.arange(0,nsemiblocks_task1,1):
                semiblock_time_task1 = semiblock_time_task1+(semiblocks_period_task1[isemiblock,1]-semiblocks_period_task1[isemiblock,0])
            blocktime_task1_all_dates[idate] = semiblock_time_task1
            #
            semiblock_time_task2 = 0
            for isemiblock in np.arange(0,nsemiblocks_task2,1):
                semiblock_time_task2 = semiblock_time_task2+(semiblocks_period_task2[isemiblock,1]-semiblocks_period_task2[isemiblock,0])
            blocktime_task2_all_dates[idate] = semiblock_time_task2    
                
                
        # analyze behavior results
        # succ_rate_all_dates[idate] = np.sum(trial_record_clean["rewarded"]>0)/np.shape(trial_record_clean)[0]
        succ_rate_all_dates[idate] = np.sum((bhv_data['behavior_events']==3)|(bhv_data['behavior_events']==4))/np.sum((bhv_data['behavior_events']==1)|(bhv_data['behavior_events']==2))
        trialnum_all_dates[idate] = np.shape(trial_record_clean)[0]
        #
        pullid = np.array(bhv_data[(bhv_data['behavior_events']==1) | (bhv_data['behavior_events']==2)]["behavior_events"])
        pulltime = np.array(bhv_data[(bhv_data['behavior_events']==1) | (bhv_data['behavior_events']==2)]["time_points"])
        pullid_diff = np.abs(pullid[1:] - pullid[0:-1])
        pulltime_diff = pulltime[1:] - pulltime[0:-1]
        interpull_intv = pulltime_diff[pullid_diff==1]
        interpull_intv = interpull_intv[interpull_intv<10]
        mean_interpull_intv = np.nanmean(interpull_intv)
        std_interpull_intv = np.nanstd(interpull_intv)
        #
        interpullintv_all_dates[idate] = mean_interpull_intv
        # 
        if animal1 == animal1_fixedorder[0]:
            pull1_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==1) 
            pull2_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==2)
        if animal1 == animal2_fixedorder[0]:
            pull1_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==2) 
            pull2_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==1)
            
        #
        if ifAlterTask:
            if animal1 == animal1_fixedorder[0]:
                pull1_num_task1_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task1)]['behavior_events']==1)
                pull2_num_task1_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task1)]['behavior_events']==2)
                pull1_num_task2_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task2)]['behavior_events']==1)
                pull2_num_task2_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task2)]['behavior_events']==2)
            if animal1 == animal2_fixedorder[0]:
                pull1_num_task1_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task1)]['behavior_events']==2)
                pull2_num_task1_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task1)]['behavior_events']==1)
                pull1_num_task2_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task2)]['behavior_events']==2)
                pull2_num_task2_all_dates[idate] = np.sum(bhv_data[np.isin(bhv_data['trial_number'],trialnum_task2)]['behavior_events']==1)
            
            
        # load behavioral event results
        try:
            dummy
            print('load social gaze with Anipose 3d of '+date_tgt)
            with open(data_saved_folder+"bhv_events_Anipose/"+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+date_tgt+'/output_look_ornot.pkl', 'rb') as f:
                output_look_ornot = pickle.load(f)
            with open(data_saved_folder+"bhv_events_Anipose/"+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+date_tgt+'/output_allvectors.pkl', 'rb') as f:
                output_allvectors = pickle.load(f)
            with open(data_saved_folder+"bhv_events_Anipose/"+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+date_tgt+'/output_allangles.pkl', 'rb') as f:
                output_allangles = pickle.load(f)  
        except:   
            print('analyze social gaze with Anipose 3d only of '+date_tgt)
            # get social gaze information 
            output_look_ornot, output_allvectors, output_allangles = find_socialgaze_timepoint_Anipose(body_part_locs_Anipose,min_length,angle_thres,with_tubelever)
            
            # save data
            current_dir = data_saved_folder+'/bhv_events_Anipose/'+animal1_fixedorder[0]+animal2_fixedorder[0]
            add_date_dir = os.path.join(current_dir+'/'+date_tgt)
            if not os.path.exists(add_date_dir):
                os.makedirs(add_date_dir)
            #
            with open(data_saved_folder+"bhv_events_Anipose/"+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+date_tgt+'/output_look_ornot.pkl', 'wb') as f:
                pickle.dump(output_look_ornot, f)
            with open(data_saved_folder+"bhv_events_Anipose/"+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+date_tgt+'/output_allvectors.pkl', 'wb') as f:
                pickle.dump(output_allvectors, f)
            with open(data_saved_folder+"bhv_events_Anipose/"+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+date_tgt+'/output_allangles.pkl', 'wb') as f:
                pickle.dump(output_allangles, f)
  

        look_at_face_or_not_Anipose = output_look_ornot['look_at_face_or_not_Anipose']
        look_at_selftube_or_not_Anipose = output_look_ornot['look_at_selftube_or_not_Anipose']
        look_at_selflever_or_not_Anipose = output_look_ornot['look_at_selflever_or_not_Anipose']
        look_at_othertube_or_not_Anipose = output_look_ornot['look_at_othertube_or_not_Anipose']
        look_at_otherlever_or_not_Anipose = output_look_ornot['look_at_otherlever_or_not_Anipose']
        # change the unit to second
        session_start_time = session_start_times[idate]
        look_at_face_or_not_Anipose['time_in_second'] = np.arange(0,np.shape(look_at_face_or_not_Anipose['dodson'])[0],1)/fps - session_start_time
        look_at_selflever_or_not_Anipose['time_in_second'] = np.arange(0,np.shape(look_at_selflever_or_not_Anipose['dodson'])[0],1)/fps - session_start_time
        look_at_selftube_or_not_Anipose['time_in_second'] = np.arange(0,np.shape(look_at_selftube_or_not_Anipose['dodson'])[0],1)/fps - session_start_time 
        look_at_otherlever_or_not_Anipose['time_in_second'] = np.arange(0,np.shape(look_at_otherlever_or_not_Anipose['dodson'])[0],1)/fps - session_start_time
        look_at_othertube_or_not_Anipose['time_in_second'] = np.arange(0,np.shape(look_at_othertube_or_not_Anipose['dodson'])[0],1)/fps - session_start_time 

        look_at_Anipose = {"face":look_at_face_or_not_Anipose,"selflever":look_at_selflever_or_not_Anipose,
                           "selftube":look_at_selftube_or_not_Anipose,"otherlever":look_at_otherlever_or_not_Anipose,
                           "othertube":look_at_othertube_or_not_Anipose} 

        # find time point of behavioral events
        output_time_points_socialgaze ,output_time_points_levertube = bhv_events_timepoint_Anipose(bhv_data,look_at_Anipose)
        time_point_pull1 = output_time_points_socialgaze['time_point_pull1']
        time_point_pull2 = output_time_points_socialgaze['time_point_pull2']
        oneway_gaze1 = output_time_points_socialgaze['oneway_gaze1']
        oneway_gaze2 = output_time_points_socialgaze['oneway_gaze2']
        mutual_gaze1 = output_time_points_socialgaze['mutual_gaze1']
        mutual_gaze2 = output_time_points_socialgaze['mutual_gaze2']
        timepoint_lever1 = output_time_points_levertube['time_point_lookatlever1']   
        timepoint_lever2 = output_time_points_levertube['time_point_lookatlever2']   
        timepoint_tube1 = output_time_points_levertube['time_point_lookattube1']   
        timepoint_tube2 = output_time_points_levertube['time_point_lookattube2']   

        #
        owgaze1_num_all_dates[idate] = np.shape(oneway_gaze1)[0]
        owgaze2_num_all_dates[idate] = np.shape(oneway_gaze2)[0]
        mtgaze1_num_all_dates[idate] = np.shape(mutual_gaze1)[0]
        mtgaze2_num_all_dates[idate] = np.shape(mutual_gaze2)[0]
         
            
        if 1:        
            # # plot behavioral events
            if np.isin(animal1,animal1_fixedorder):
                    plot_bhv_events(date_tgt,animal1, animal2, session_start_time, totalsess_time, time_point_pull1, time_point_pull2, oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2)
            else:
                    plot_bhv_events(date_tgt,animal2, animal1, session_start_time, totalsess_time, time_point_pull2, time_point_pull1, oneway_gaze2, oneway_gaze1, mutual_gaze2, mutual_gaze1)
            #
            # save behavioral events plot       
            current_dir = data_saved_folder+'/bhv_events_Anipose/'+animal1_fixedorder[0]+animal2_fixedorder[0]
            add_date_dir = os.path.join(current_dir+'/'+date_tgt)
            if not os.path.exists(add_date_dir):
                os.makedirs(add_date_dir)
            plt.savefig(data_saved_folder+"/bhv_events_Anipose/"+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+date_tgt+'/'+date_tgt+"_"+".pdf")

        #
        if animal1 == animal1_fixedorder[0]:
            owgaze1_num_all_dates[idate] = np.shape(oneway_gaze1)[0]
            owgaze2_num_all_dates[idate] = np.shape(oneway_gaze2)[0]
            mtgaze1_num_all_dates[idate] = np.shape(mutual_gaze1)[0]
            mtgaze2_num_all_dates[idate] = np.shape(mutual_gaze2)[0]
        if animal1 == animal2_fixedorder[0]:
            owgaze1_num_all_dates[idate] = np.shape(oneway_gaze2)[0]
            owgaze2_num_all_dates[idate] = np.shape(oneway_gaze1)[0]
            mtgaze1_num_all_dates[idate] = np.shape(mutual_gaze2)[0]
            mtgaze2_num_all_dates[idate] = np.shape(mutual_gaze1)[0]
        #
        if ifAlterTask:
            # task 1
            owgaze1_num_task1 = 0
            mtgaze1_num_task1 = 0
            owgaze2_num_task1 = 0
            mtgaze2_num_task1 = 0
            for isemiblock in np.arange(0,nsemiblocks_task1,1):
                owgaze1_num_task1 = owgaze1_num_task1+np.sum((oneway_gaze1>semiblocks_period_task1[isemiblock,0])&(oneway_gaze1<semiblocks_period_task1[isemiblock,1]))
                mtgaze1_num_task1 = mtgaze1_num_task1+np.sum((mutual_gaze1>semiblocks_period_task1[isemiblock,0])&(mutual_gaze1<semiblocks_period_task1[isemiblock,1]))
                owgaze2_num_task1 = owgaze2_num_task1+np.sum((oneway_gaze2>semiblocks_period_task1[isemiblock,0])&(oneway_gaze2<semiblocks_period_task1[isemiblock,1]))
                mtgaze2_num_task1 = mtgaze2_num_task1+np.sum((mutual_gaze2>semiblocks_period_task1[isemiblock,0])&(mutual_gaze2<semiblocks_period_task1[isemiblock,1]))
            if animal1 == animal1_fixedorder[0]:
                owgaze1_num_task1_all_dates[idate] = owgaze1_num_task1
                owgaze2_num_task1_all_dates[idate] = owgaze2_num_task1
                mtgaze1_num_task1_all_dates[idate] = mtgaze1_num_task1
                mtgaze2_num_task1_all_dates[idate] = mtgaze2_num_task1 
            if animal1 == animal2_fixedorder[0]:
                owgaze1_num_task1_all_dates[idate] = owgaze2_num_task1
                owgaze2_num_task1_all_dates[idate] = owgaze1_num_task1
                mtgaze1_num_task1_all_dates[idate] = mtgaze2_num_task1
                mtgaze2_num_task1_all_dates[idate] = mtgaze1_num_task1 
            # task 2
            owgaze1_num_task2 = 0
            mtgaze1_num_task2 = 0
            owgaze2_num_task2 = 0
            mtgaze2_num_task2 = 0
            for isemiblock in np.arange(0,nsemiblocks_task2,1):
                owgaze1_num_task2 = owgaze1_num_task2+np.sum((oneway_gaze1>semiblocks_period_task2[isemiblock,0])&(oneway_gaze1<semiblocks_period_task2[isemiblock,1]))
                mtgaze1_num_task2 = mtgaze1_num_task2+np.sum((mutual_gaze1>semiblocks_period_task2[isemiblock,0])&(mutual_gaze1<semiblocks_period_task2[isemiblock,1]))
                owgaze2_num_task2 = owgaze2_num_task2+np.sum((oneway_gaze2>semiblocks_period_task2[isemiblock,0])&(oneway_gaze2<semiblocks_period_task2[isemiblock,1]))
                mtgaze2_num_task2 = mtgaze2_num_task2+np.sum((mutual_gaze2>semiblocks_period_task2[isemiblock,0])&(mutual_gaze2<semiblocks_period_task2[isemiblock,1]))
            if animal1 == animal1_fixedorder[0]:
                owgaze1_num_task2_all_dates[idate] = owgaze1_num_task2
                owgaze2_num_task2_all_dates[idate] = owgaze2_num_task2
                mtgaze1_num_task2_all_dates[idate] = mtgaze1_num_task2
                mtgaze2_num_task2_all_dates[idate] = mtgaze2_num_task2  
            if animal1 == animal2_fixedorder[0]:
                owgaze1_num_task2_all_dates[idate] = owgaze2_num_task2
                owgaze2_num_task2_all_dates[idate] = owgaze1_num_task2
                mtgaze1_num_task2_all_dates[idate] = mtgaze2_num_task2
                mtgaze2_num_task2_all_dates[idate] = mtgaze1_num_task2  

        

        # analyze the events interval, especially for the pull to other and other to pull interval
        # could be used for define time bin for DBN
        if 1:
            _,_,_,pullTOother_itv, otherTOpull_itv = bhv_events_interval(totalsess_time, session_start_time, time_point_pull1, time_point_pull2, 
                                                                         oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2)
            #
            pull_other_pool_itv = np.concatenate((pullTOother_itv,otherTOpull_itv))
            bhv_intv_all_dates[date_tgt] = {'pull_to_other':pullTOother_itv,'other_to_pull':otherTOpull_itv,
                            'pull_other_pooled': pull_other_pool_itv}
        
        # plot the tracking demo video
        if 1:      
            video_file = video_file_dir+'/'+date_tgt+'_'+animal1_filename+'_'+animal2_filename+'_anipose_bhv_demo.mp4'
            tracking_video_Anipose_events_demo(body_part_locs_Anipose,output_look_ornot,output_allvectors,output_allangles,
                                           time_point_pull1,time_point_pull2,animalnames_videotrack,bodypartnames_videotrack,
                                           date_tgt,animal1_filename,animal2_filename,animal1,animal2,
                                           session_start_time,fps,nframes,video_file,withboxCorner)

    # save data
    if 1:
        
        data_saved_subfolder = data_saved_folder+'data_saved_Anipose'+savefile_sufix+'/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'
        if not os.path.exists(data_saved_subfolder):
            os.makedirs(data_saved_subfolder)
                
        # with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
        #     pickle.dump(DBN_input_data_alltypes, f)

        with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze1_num_all_dates, f)
        with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze2_num_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze1_num_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze2_num_all_dates, f)
        with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull1_num_all_dates, f)
        with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull2_num_all_dates, f)
        with open(data_saved_subfolder+'/totalsess_time_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(totalsess_time_all_dates, f)
        with open(data_saved_subfolder+'/blocktime_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(blocktime_all_dates, f)
        with open(data_saved_subfolder+'/animal1_blocked_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(animal1_blocked_all_dates, f)
        with open(data_saved_subfolder+'/animal2_blocked_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(animal2_blocked_all_dates, f)
        #
        with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(succ_rate_all_dates, f)
        with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(interpullintv_all_dates, f)
        with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(trialnum_all_dates, f)
        with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(bhv_intv_all_dates, f)
    
        # task 1
        with open(data_saved_subfolder+'/trialnum_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(trialnum_task1_all_dates, f)
        with open(data_saved_subfolder+'/pull1_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull1_num_task1_all_dates, f)
        with open(data_saved_subfolder+'/pull2_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull2_num_task1_all_dates, f)
        with open(data_saved_subfolder+'/owgaze1_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze1_num_task1_all_dates, f)
        with open(data_saved_subfolder+'/owgaze2_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze2_num_task1_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze1_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze1_num_task1_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze2_num_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze2_num_task1_all_dates, f)  
        with open(data_saved_subfolder+'/blocktime_task1_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(blocktime_task1_all_dates, f)
        
        # task 2
        with open(data_saved_subfolder+'/trialnum_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(trialnum_task2_all_dates, f)
        with open(data_saved_subfolder+'/pull1_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull1_num_task2_all_dates, f)
        with open(data_saved_subfolder+'/pull2_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(pull2_num_task2_all_dates, f)
        with open(data_saved_subfolder+'/owgaze1_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze1_num_task2_all_dates, f)
        with open(data_saved_subfolder+'/owgaze2_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze2_num_task2_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze1_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze1_num_task2_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze2_num_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze2_num_task2_all_dates, f)  
        with open(data_saved_subfolder+'/blocktime_task2_all_dates_'+animal1_fixedorder[0]+animal2_fixedorder[0]+'.pkl', 'wb') as f:
            pickle.dump(blocktime_task2_all_dates, f)
            
            

## figure plot
### plot the gaze number/block time; task 1 vs task 2

In [ ]:
fig, axs = plt.subplots(3,3)
fig.set_figheight(3*8)
fig.set_figwidth(3*8)

ind_norm = (animal1_blocked_all_dates+animal2_blocked_all_dates)==0
ind_blockedA1 = animal1_blocked_all_dates == 1
ind_blockedA2 = animal2_blocked_all_dates == 1

types = ['normal','Koala blocked','Vermelho blocked']

for itype in np.arange(0,3,1):
    
    if itype == 0:
        ind_plot = ind_norm
    elif itype == 1:
        ind_plot = ind_blockedA1
    elif itype == 2:
        ind_plot = ind_blockedA2
        
    typenames = types[itype]
        
    # animal 1 gaze 
    xxx = (mtgaze1_num_task1_all_dates[ind_plot]+owgaze1_num_task1_all_dates[ind_plot])/blocktime_task1_all_dates[ind_plot]
    yyy = (mtgaze1_num_task2_all_dates[ind_plot]+owgaze1_num_task2_all_dates[ind_plot])/blocktime_task2_all_dates[ind_plot]
    axs[itype,0].plot(xxx,yyy,'ko')
    axs[itype,0].plot([0,5],[0,5],'k--')
    axs[itype,0].set_xlim([0,5])
    axs[itype,0].set_ylim([0,5])
    axs[itype,0].set_aspect('equal', adjustable='box')
    if itype == 2:
        axs[itype,0].set_xlabel('social gaze number/second; task 1',fontsize = 15)
    axs[itype,0].set_ylabel('social gaze number/second; task 2',fontsize = 15)
    sts = scipy.stats.ttest_rel(xxx,yyy)
    pvalue = sts.pvalue
    axs[itype,0].text(1,0.25,'ttest p='+"{:.3f}".format(pvalue),fontsize=13)
    axs[itype,0].set_title(typenames+' sessions\n'+animal1_fixedorder[0],fontsize=16)

    # animal 2 gaze 
    xxx = (mtgaze2_num_task1_all_dates[ind_plot]+owgaze2_num_task1_all_dates[ind_plot])/blocktime_task1_all_dates[ind_plot]
    yyy = (mtgaze2_num_task2_all_dates[ind_plot]+owgaze2_num_task2_all_dates[ind_plot])/blocktime_task2_all_dates[ind_plot]
    axs[itype,1].plot(xxx,yyy,'ko')
    axs[itype,1].plot([0,5],[0,5],'k--')
    axs[itype,1].set_xlim([0,5])
    axs[itype,1].set_ylim([0,5])
    axs[itype,1].set_aspect('equal', adjustable='box')
    if itype == 2:
        axs[itype,1].set_xlabel('social gaze number/second; task 1',fontsize = 15)
    sts = scipy.stats.ttest_rel(xxx,yyy)
    pvalue = sts.pvalue
    axs[itype,1].text(1,0.25,'ttest p='+"{:.3f}".format(pvalue),fontsize=13)
    axs[itype,1].set_title(typenames+' sessions\n'+animal2_fixedorder[0],fontsize=16)


    # both animals gaze 
    xxx1 = (mtgaze1_num_task1_all_dates[ind_plot]+owgaze1_num_task1_all_dates[ind_plot])/blocktime_task1_all_dates[ind_plot]
    yyy1 = (mtgaze1_num_task2_all_dates[ind_plot]+owgaze1_num_task2_all_dates[ind_plot])/blocktime_task2_all_dates[ind_plot]
    xxx2 = (mtgaze2_num_task1_all_dates[ind_plot]+owgaze2_num_task1_all_dates[ind_plot])/blocktime_task1_all_dates[ind_plot]
    yyy2 = (mtgaze2_num_task2_all_dates[ind_plot]+owgaze2_num_task2_all_dates[ind_plot])/blocktime_task2_all_dates[ind_plot]
    xxx = np.hstack([xxx1,xxx2])
    yyy = np.hstack([yyy1,yyy2])
    axs[itype,2].plot(xxx,yyy,'ko')
    axs[itype,2].plot([0,5],[0,5],'k--')
    axs[itype,2].set_xlim([0,5])
    axs[itype,2].set_ylim([0,5])
    axs[itype,2].set_aspect('equal', adjustable='box')
    if itype == 2:
        axs[itype,2].set_xlabel('social gaze number/second time; task 1',fontsize = 15)
    sts = scipy.stats.ttest_rel(xxx,yyy)
    pvalue = sts.pvalue
    axs[itype,2].text(1,0.25,'ttest p='+"{:.3f}".format(pvalue),fontsize=13)
    axs[itype,2].set_title(typenames+' sessions\n'+'Combine two animals',fontsize=16)


savefigs = 1
if savefigs:
    plt.savefig("gazenum_perblocktime_task1VStask2"+animal1_fixedorder[0]+animal2_fixedorder[0]+"_Anipose.pdf")